In [1]:
import random
import time

In [85]:
class TennerGridCSP:
    def __init__(self, rows, columns, target_sums):
        self.rows = rows
        self.columns = columns
        self.target_sums = target_sums
        self.grid = [[None for _ in range(columns)] for _ in range(rows)]

    def is_valid_assignment(self, row, col, value):
        # Check row uniqueness
        if value in self.grid[row]:
            return False

        # Check column sum constraint
        col_sum = sum(self.grid[r][col] for r in range(self.rows) if self.grid[r][col] is not None)
        if col_sum + value > self.target_sums[col]:
            return False

        # Check contiguous cells constraint
        neighbors = [
            (row - 1, col - 1), (row - 1, col), (row - 1, col + 1),
            (row, col - 1), (row, col + 1),
            (row + 1, col - 1), (row + 1, col), (row + 1, col + 1)
        ]
        for neighbor_row, neighbor_col in neighbors:
            if 0 <= neighbor_row < self.rows and 0 <= neighbor_col < self.columns:
                neighbor_value = self.grid[neighbor_row][neighbor_col]
                if neighbor_value == value:
                    return False

        # Check target sum prediction
        for c in range(self.columns):
            if c != col:
                col_sum = sum(self.grid[r][c] for r in range(self.rows) if self.grid[r][c] is not None)
                if col_sum + value > self.target_sums[c]:
                    return False

        return True
    def is_complete(self):
        return all(None not in row for row in self.grid)

    def get_unassigned_variable(self):
        for row in range(self.rows):
            for col in range(self.columns):
                if self.grid[row][col] is None:
                    return row, col
        return None

    def simple_backtracking(self):
        assignments = 0
        checks = 0

        def backtrack():
            nonlocal assignments, checks

            if self.is_complete():
                return True

            row, col = self.get_unassigned_variable()
            for value in range(10):
                checks += 1
                if self.is_valid_assignment(row, col, value):
                    self.grid[row][col] = value
                    assignments += 1
                    if backtrack():
                        return True
                    self.grid[row][col] = None

            return False

        start_time = time.time()
        backtrack()
        end_time = time.time()

        return self.grid, assignments, checks, end_time - start_time

    def mrv_heuristic(self):
        assignments = 0
        checks = 0

        def get_mrv_variable():
            min_remaining_values = float('inf')
            mrv_variable = None

            for row in range(self.rows):
                for col in range(self.columns):
                    if self.grid[row][col] is None:
                        remaining_values = 10 - len(set(self.grid[row]))
                        if remaining_values < min_remaining_values:
                            min_remaining_values = remaining_values
                            mrv_variable = (row, col)

            return mrv_variable

        def backtrack():
            nonlocal assignments, checks

            if self.is_complete():
                return True

            row, col = get_mrv_variable()
            for value in range(10):
                checks += 1
                if self.is_valid_assignment(row, col, value):
                    self.grid[row][col] = value
                    assignments += 1
                    if backtrack():
                        return True
                    self.grid[row][col] = None

            return False

        start_time = time.time()
        backtrack()
        end_time = time.time()

        return self.grid, assignments, checks, end_time - start_time

    def forward_checking(self):
        assignments = 0
        checks = 0

        def forward_check(row, col):
            nonlocal checks

            for r in range(row + 1, self.rows):
                domain = set(self.grid[r])
                checks += 1
                if self.grid[r][col] is not None:
                    domain.remove(self.grid[r][col])
                if len(domain) == 0:
                    return False

            return True

        def backtrack():
            nonlocal assignments, checks

            if self.is_complete():
                return True

            row, col = self.get_unassigned_variable()
            for value in range(10):
                checks += 1
                if self.is_valid_assignment(row, col, value):
                    self.grid[row][col] = value
                    assignments += 1
                    if forward_check(row, col) and backtrack():
                        return True
                    self.grid[row][col] = None

            return False

        start_time = time.time()
        backtrack()
        end_time = time.time()

        return self.grid, assignments, checks, end_time - start_time

    def forward_checking_mrv(self):
        assignments = 0
        checks = 0

        def get_mrv_variable():
            min_remaining_values = float('inf')
            mrv_variable = None

            for row in range(self.rows):
                for col in range(self.columns):
                    if self.grid[row][col] is None:
                        remaining_values = 10 - len(set(self.grid[row]))
                        if remaining_values < min_remaining_values:
                            min_remaining_values = remaining_values
                            mrv_variable = (row, col)

            return mrv_variable

        def forward_check(row, col):
            nonlocal checks

            for r in range(row + 1, self.rows):
                domain = set(self.grid[r])
                checks += 1
                if self.grid[r][col] is not None:
                    domain.remove(self.grid[r][col])
                if len(domain) == 0:
                    return False

            return True

        def backtrack():
            nonlocal assignments, checks

            if self.is_complete():
                return True

            row, col = get_mrv_variable()
            for value in range(10):
                checks += 1
                if self.is_valid_assignment(row, col, value):
                    self.grid[row][col] = value
                    assignments += 1
                    if forward_check(row, col) and backtrack():
                        return True
                    self.grid[row][col] = None

            return False

        start_time = time.time()
        backtrack()
        end_time = time.time()

        return self.grid, assignments, checks, end_time - start_time

    def print_solution(self):
        for row in self.grid:
            print(row)



In [86]:
rows = 3
columns = 2
target_sums = [5,12]

In [87]:
# Create the TennerGridCSP object
tenner_grid = TennerGridCSP(rows, columns, target_sums)

# Solve using Simple Backtracking
print("Simple Backtracking:")
grid, assignments, checks, time_taken = tenner_grid.simple_backtracking()
tenner_grid.print_solution()
print("Variable Assignments:", assignments)
print("Consistency Checks:", checks)
print("Time Taken:", time_taken, "seconds")

Simple Backtracking:
[0, 1]
[2, 3]
[0, 1]
Variable Assignments: 6
Consistency Checks: 13
Time Taken: 0.00011324882507324219 seconds


In [88]:
# Solve using Backtracking with MRV Heuristic
print("\nBacktracking with MRV Heuristic:")
grid, assignments, checks, time_taken = tenner_grid.mrv_heuristic()
tenner_grid.print_solution()
print("Variable Assignments:", assignments)
print("Consistency Checks:", checks)
print("Time Taken:", time_taken, "seconds")


Backtracking with MRV Heuristic:
[0, 1]
[2, 3]
[0, 1]
Variable Assignments: 0
Consistency Checks: 0
Time Taken: 5.9604644775390625e-06 seconds


In [89]:
# Solve using Forward Checking
print("\nForward Checking:")
grid, assignments, checks, time_taken = tenner_grid.forward_checking()
tenner_grid.print_solution()
print("Variable Assignments:", assignments)
print("Consistency Checks:", checks)
print("Time Taken:", time_taken, "seconds")


Forward Checking:
[0, 1]
[2, 3]
[0, 1]
Variable Assignments: 0
Consistency Checks: 0
Time Taken: 4.76837158203125e-06 seconds


In [90]:
# Solve using Forward Checking with MRV Heuristic
print("\nForward Checking with MRV Heuristic:")
grid, assignments, checks, time_taken = tenner_grid.forward_checking_mrv()
tenner_grid.print_solution()
print("Variable Assignments:", assignments)
print("Consistency Checks:", checks)
print("Time Taken:", time_taken, "seconds")



Forward Checking with MRV Heuristic:
[0, 1]
[2, 3]
[0, 1]
Variable Assignments: 0
Consistency Checks: 0
Time Taken: 5.9604644775390625e-06 seconds
